In [1]:
# lets make the client code
import socket,cv2, pickle,struct,threading,imutils
# create socket
client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
client_socket.setsockopt(socket.SOL_SOCKET , socket.SO_REUSEADDR, 1)
host_ip = '192.168.56.1' # paste your server ip address here
port = 1234
client_socket.connect((host_ip,port)) # a tuple
def receving_from_server():
    data = b""
    payload_size = struct.calcsize("Q")
    while True:
        while len(data) < payload_size:
            packet = client_socket.recv(4*1024) # 4K
            if not packet: break
            data+=packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q",packed_msg_size)[0]

        while len(data) < msg_size:
            data += client_socket.recv(4*1024)
        frame_data = data[:msg_size]
        data  = data[msg_size:]
        frame = pickle.loads(frame_data)
        cv2.imshow("Server to Client VIDEO",frame)
        key = cv2.waitKey(1) & 0xFF
        if key  == 13:
            client_socket.close()
            break
    cv2.destroyAllWindows()
def sending_client():
    if client_socket:
        vid = cv2.VideoCapture("http://192.168.43.1:8080/video")
        
        
        while(vid.isOpened()):
            try:
                img,frame = vid.read()
                frame = imutils.resize(frame,width=520)
                a = pickle.dumps(frame)
                message = struct.pack("Q",len(a))+a
                client_socket.sendall(message)
            except ConnectionAbortedError:
                vid.release()
                client_socket.close()
                break
t1 = threading.Thread(target=receving_from_server)
t1.start()
t2 = threading.Thread(target=sending_client)
t2.start()